<a href="https://colab.research.google.com/github/eswens13/deep_learning/blob/master/cifar_10_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CIFAR-10 Classifier/Autoencoder

This notebook is an exploratory exercise in convolutional neural networks.  I will build a classifier for the CIFAR-10 image set and play around with network architecture, hyperparameters, visualization techniques, etc. to get hands-on experience coding convolutional neural networks in TensorFlow.

I will also explore the differences between a classifier and an auto-encoder.

## Define Network Architecture

First, we have to define a network architecture.  The code in the cell below has comments explaining the architecture of each of the layers.

In [0]:
import tensorflow as tf

def create_model(input_image):
  """
  Defines the neural network architecture for CIFAR-10 classification.
  
  Parameters:
    input_image - The image to classify. Shape: (N x 32 x 32 x 3)
  """
  input_shape = input_image.shape
  (N, H, W, C) = input_shape
  
  # Convolutional Layer 1:
  #    - Input shape: (N, 32, 32, 3)
  #    - 16 3x3 filters
  #    - Zero-pad input to keep same feature map dimensions
  #    - ReLU activation
  #    - Output shape: (N, 32, 32, 16)
  with tf.variable_scope("conv1") as scope:
    filter_shape_conv1 = (3, 3)
    filters_conv1 = 16
    img = tf.placeholder(tf.float32, shape=input_shape)
    w_conv1 = tf.get_variable("W", \
                              [filter_shape_conv1[0], \
                                filter_shape_conv1[1], \
                                C, \
                                filters_conv1], \
                              initializer=tf.random_normal_initializer)
    b_conv1 = tf.get_variable("b", \
                              [filters_conv1], \
                              initializer=tf.zeros_initializer)
    strides = [1, 1, 1, 1]
    padding = "SAME"
    
    conv_conv1 = tf.nn.conv2d(img, w_conv1, strides, padding)
    out1 = tf.nn.relu(conv_conv1 + b_conv1, name=scope.name)
  
  # Convolutional Layer 2:
  #    - Input shape: (N, 32, 32, 16)
  #    - 32 3x3 filters
  #    - Lose a pixel off each side because of convolution.
  #    - ReLU activation
  #    - Output shape: (N, 30, 30, 32)
  with tf.variable_scope("conv2") as scope:
    filter_shape_conv2 = (3, 3)
    filters_conv2 = 32
    w_conv2 = tf.get_variable("W", \
                              [filter_shape_conv2[0], \
                                filter_shape_conv2[1], \
                                filters_conv1, \
                                filters_conv2], \
                              initializer=tf.random_normal_initializer)
    b_conv2 = tf.get_variable("b", \
                              [filters_conv2], \
                              initializer=tf.zeros_initializer)
    strides = [1, 1, 1, 1]
    padding = "VALID"
    
    conv_conv2 = tf.nn.conv2d(out1, w_conv2, strides, padding)
    out2 = tf.nn.relu(conv_conv2 + b_conv2, name=scope.name)
  
  # Convolutional Layer 3:
  #    - Input shape: (N, 30, 30, 32)
  #    - 64 5x5 filters
  #    - Lose two pixels off each side because of convolution.
  #    - ReLU activation
  #    - Output shape: (N, 26, 26, 64)
  with tf.variable_scope("conv3") as scope:
    filter_shape_conv3 = (5, 5)
    filters_conv3 = 64
    w_conv3 = tf.get_variable("W", \
                              [filter_shape_conv3[0], \
                                filter_shape_conv3[1], \
                                filters_conv2, \
                                filters_conv3], \
                              initializer=tf.random_normal_initializer)
    b_conv3 = tf.get_variable("b", \
                              [filters_conv3], \
                              initializer=tf.zeros_initializer)
    strides = [1, 1, 1, 1]
    padding = "VALID"
    
    conv_conv3 = tf.nn.conv2d(out2, w_conv3, strides, padding)
    out3 = tf.nn.relu(conv_conv3 + b_conv3, name=scope.name)
  
  # Create the TF session and run the graph.
  with tf.Session() as sess:
    # Initialize all the variables according to their initializers.
    sess.run(tf.global_variables_initializer())
    
    # This call starts the chain of operations in the computation graph created
    # above.
    output = sess.run(out3, feed_dict={img: input_image})
    print("Output Shape: {}".format(output.shape))

In [34]:
import numpy as np

# Reset the graph so we don't have variable collisions when we re-run.
tf.reset_default_graph()

# Simulate a batch of 32 images (each 32 x 32 pixels RBG)
input_image = np.zeros((32, 32, 32, 3))
create_model(input_image)

Output Shape: (32, 26, 26, 64)
